In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d mohamedhanyyy/chest-ctscan-images

 95% 113M/119M [00:02<00:00, 59.7MB/s]
100% 119M/119M [00:02<00:00, 60.5MB/s]


In [5]:
import zipfile
zip_file_path = '/content/chest-ctscan-images.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [6]:
from datetime import datetime
import time

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, Dense, GlobalAveragePooling2D, BatchNormalization, Flatten

import numpy as np
import cv2
import os
import io
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import datetime
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from time import time
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, LambdaCallback
from keras.layers import Input, Dropout, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.applications.resnet import ResNet50


In [7]:
%ls /content/Data/test
%ls /content/Data/train
%ls /content/Data/valid


adenocarcinoma/  large.cell.carcinoma/  normal/  squamous.cell.carcinoma/
adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/     normal/
large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa/  squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa/
adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/     normal/
large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa/  squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa/


In [7]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Define image size
image_size = 150

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, 0)
    image = cv2.bilateralFilter(image, 2, 50, 50)
    image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
    image = cv2.resize(image, (image_size, image_size))
    return image

# Define data directories
data_dirs = {
    'train': '/content/Data/train',
    'test': '/content/Data/test',
    'valid': '/content/Data/valid'
}

# Initialize lists to store data
data = {split: {'images': [], 'labels': []} for split in data_dirs}

# Iterate over data directories
for split, directory in data_dirs.items():
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for file in tqdm(os.listdir(label_dir), desc=f'Processing {split}/{label}'):
            image_path = os.path.join(label_dir, file)
            data[split]['images'].append(preprocess_image(image_path))
            data[split]['labels'].append(label)

# Convert lists to NumPy arrays and normalize images
for split in data:
    data[split]['images'] = np.array(data[split]['images']) / 255.0
    data[split]['labels'] = np.array(data[split]['labels'])

# Extract unique labels present in the data
unique_labels = np.unique(np.concatenate([data[split]['labels'] for split in data]))

# Convert labels to one-hot encoding using dynamically extracted labels
label_to_index = {label: i for i, label in enumerate(unique_labels)}
for split in data:
    data[split]['labels'] = tf.keras.utils.to_categorical([label_to_index[label] for label in data[split]['labels']], num_classes=len(unique_labels))


# Shuffle training data
data['train']['images'], data['train']['labels'] = shuffle(data['train']['images'], data['train']['labels'], random_state=42)

# Split dataset into training, validation, and test sets
train_val_split = 0.2
val_test_split = 0.5
x_train, x_val_test, y_train, y_val_test = train_test_split(data['train']['images'], data['train']['labels'], test_size=train_val_split, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=val_test_split, random_state=42)


Processing train/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa: 100%|██████████| 155/155 [00:01<00:00, 115.98it/s]
Processing train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib: 100%|██████████| 195/195 [00:02<00:00, 94.64it/s]
Processing train/normal: 100%|██████████| 148/148 [00:03<00:00, 42.46it/s]
Processing train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa: 100%|██████████| 115/115 [00:00<00:00, 141.17it/s]
Processing test/large.cell.carcinoma: 100%|██████████| 51/51 [00:00<00:00, 113.36it/s]
Processing valid/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa: 100%|██████████| 15/15 [00:00<00:00, 140.27it/s]
Processing valid/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib: 100%|██████████| 23/23 [00:00<00:00, 115.63it/s]
Processing valid/normal: 100%|██████████| 13/13 [00:00<00:00, 39.81it/s]
Processing valid/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa: 100%|██████████| 21/21 [00:00<00:00, 184.76it/s]


In [ ]:
import time
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))


**Vgg16**

In [ ]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
# Add custom output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(unique_labels), activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model on training data
start_time = time.time()
history = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))
end_time = time.time()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)

# Print metrics
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')

Epoch 1/10
16/16 [==============================] - 30s 868ms/step - loss: 2.2023 - accuracy: 0.2265 - val_loss: 1.7030 - val_accuracy: 0.2951
Epoch 2/10
16/16 [==============================] - 3s 196ms/step - loss: 1.4107 - accuracy: 0.2898 - val_loss: 1.4114 - val_accuracy: 0.2295
Epoch 3/10
16/16 [==============================] - 3s 196ms/step - loss: 1.3035 - accuracy: 0.3959 - val_loss: 1.2850 - val_accuracy: 0.4098
Epoch 4/10
16/16 [==============================] - 3s 197ms/step - loss: 1.0961 - accuracy: 0.4694 - val_loss: 1.0431 - val_accuracy: 0.4426
Epoch 5/10
16/16 [==============================] - 3s 202ms/step - loss: 0.9949 - accuracy: 0.5122 - val_loss: 1.2600 - val_accuracy: 0.3934
Epoch 6/10
16/16 [==============================] - 3s 200ms/step - loss: 1.0806 - accuracy: 0.4755 - val_loss: 1.0730 - val_accuracy: 0.5410
Epoch 7/10
16/16 [==============================] - 3s 199ms/step - loss: 0.9616 - accuracy: 0.5612 - val_loss: 0.9382 - val_accuracy: 0.5082
Epoch

**InceptionV3**

In [ ]:

import time
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))


# Add custom output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(unique_labels), activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model on training data
start_time = time.time()
history = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))
end_time = time.time()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)

# Print metrics
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')

Epoch 1/10
16/16 [==============================] - 49s 653ms/step - loss: 1.2042 - accuracy: 0.5653 - val_loss: 3.7733 - val_accuracy: 0.3443
Epoch 2/10
16/16 [==============================] - 2s 146ms/step - loss: 0.3508 - accuracy: 0.9184 - val_loss: 4.1972 - val_accuracy: 0.2787
Epoch 3/10
16/16 [==============================] - 2s 110ms/step - loss: 0.1106 - accuracy: 0.9857 - val_loss: 2.8532 - val_accuracy: 0.3279
Epoch 4/10
16/16 [==============================] - 2s 110ms/step - loss: 0.0433 - accuracy: 0.9898 - val_loss: 1.9943 - val_accuracy: 0.4426
Epoch 5/10
16/16 [==============================] - 2s 122ms/step - loss: 0.0350 - accuracy: 0.9918 - val_loss: 1.5689 - val_accuracy: 0.4918
Epoch 6/10
16/16 [==============================] - 2s 112ms/step - loss: 0.0327 - accuracy: 0.9959 - val_loss: 1.2874 - val_accuracy: 0.6066
Epoch 7/10
16/16 [==============================] - 2s 111ms/step - loss: 0.0208 - accuracy: 0.9959 - val_loss: 1.0666 - val_accuracy: 0.6393
Epoch

**Vgg19**

In [ ]:
from tensorflow.keras.applications import VGG19

# Load pre-trained VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Add custom output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(unique_labels), activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model on training data
start_time = time.time()
history = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))
end_time = time.time()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)

# Print metrics
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')

Epoch 1/10
16/16 [==============================] - 10s 293ms/step - loss: 1.3351 - accuracy: 0.4408 - val_loss: 1.2242 - val_accuracy: 0.3443
Epoch 2/10
16/16 [==============================] - 4s 266ms/step - loss: 1.0233 - accuracy: 0.5184 - val_loss: 0.9702 - val_accuracy: 0.5738
Epoch 3/10
16/16 [==============================] - 4s 268ms/step - loss: 0.8208 - accuracy: 0.6306 - val_loss: 0.7541 - val_accuracy: 0.6393
Epoch 4/10
16/16 [==============================] - 4s 270ms/step - loss: 0.6049 - accuracy: 0.7408 - val_loss: 0.6525 - val_accuracy: 0.6721
Epoch 5/10
16/16 [==============================] - 5s 284ms/step - loss: 0.5059 - accuracy: 0.7939 - val_loss: 0.4657 - val_accuracy: 0.7541
Epoch 6/10
16/16 [==============================] - 4s 272ms/step - loss: 0.3044 - accuracy: 0.8816 - val_loss: 0.5353 - val_accuracy: 0.7705
Epoch 7/10
16/16 [==============================] - 4s 269ms/step - loss: 0.3280 - accuracy: 0.8796 - val_loss: 0.2745 - val_accuracy: 0.8852
Epoch

**apply noise**

In [8]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Define image size
image_size = 150

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, 0)
    image = cv2.bilateralFilter(image, 2, 50, 50)
    image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
    image = cv2.resize(image, (image_size, image_size))
    return image

# Define data directories
data_dirs = {
    'train': '/content/Data/train',
    'test': '/content/Data/test',
    'valid': '/content/Data/valid'
}

# Initialize lists to store data
data = {split: {'images': [], 'labels': []} for split in data_dirs}

# Iterate over data directories
for split, directory in data_dirs.items():
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for file in tqdm(os.listdir(label_dir), desc=f'Processing {split}/{label}'):
            image_path = os.path.join(label_dir, file)
            data[split]['images'].append(preprocess_image(image_path))
            data[split]['labels'].append(label)

# Convert lists to NumPy arrays and normalize images
for split in data:
    data[split]['images'] = np.array(data[split]['images']) / 255.0
    data[split]['labels'] = np.array(data[split]['labels'])

# Extract unique labels present in the data
unique_labels = np.unique(np.concatenate([data[split]['labels'] for split in data]))

# Convert labels to one-hot encoding using dynamically extracted labels
label_to_index = {label: i for i, label in enumerate(unique_labels)}
for split in data:
    data[split]['labels'] = tf.keras.utils.to_categorical([label_to_index[label] for label in data[split]['labels']], num_classes=len(unique_labels))

# Shuffle training data
data['train']['images'], data['train']['labels'] = shuffle(data['train']['images'], data['train']['labels'], random_state=42)

# Split dataset into training, validation, and test sets
train_val_split = 0.2
val_test_split = 0.5
x_train, x_val_test, y_train, y_val_test = train_test_split(data['train']['images'], data['train']['labels'], test_size=train_val_split, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=val_test_split, random_state=42)

# Apply data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
datagen.fit(x_train)

Processing train/normal: 100%|██████████| 148/148 [00:01<00:00, 80.85it/s]
Processing train/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa: 100%|██████████| 155/155 [00:00<00:00, 168.06it/s]
Processing train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa: 100%|██████████| 115/115 [00:00<00:00, 173.05it/s]
Processing train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib: 100%|██████████| 195/195 [00:01<00:00, 169.33it/s]
Processing valid/normal: 100%|██████████| 13/13 [00:00<00:00, 57.55it/s]
Processing valid/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa: 100%|██████████| 15/15 [00:00<00:00, 181.07it/s]
Processing valid/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa: 100%|██████████| 21/21 [00:00<00:00, 193.70it/s]
Processing valid/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib: 100%|██████████| 23/23 [00:00<00:00, 187.89it/s]


**Vgg16**

In [ ]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
# Add custom output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(unique_labels), activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model on training data
start_time = time.time()
batch_size = 32
epochs = 10
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=len(x_train) // batch_size, epochs=epochs, validation_data=(x_val, y_val))
end_time = time.time()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)

# Print metrics
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')

Epoch 1/10
15/15 [==============================] - 8s 288ms/step - loss: 4.1279 - accuracy: 0.2314 - val_loss: 1.5649 - val_accuracy: 0.2951
Epoch 2/10
15/15 [==============================] - 4s 233ms/step - loss: 1.4518 - accuracy: 0.2817 - val_loss: 1.4457 - val_accuracy: 0.2951
Epoch 3/10
15/15 [==============================] - 5s 316ms/step - loss: 1.3442 - accuracy: 0.3166 - val_loss: 1.2323 - val_accuracy: 0.4590
Epoch 4/10
15/15 [==============================] - 4s 258ms/step - loss: 1.2064 - accuracy: 0.3886 - val_loss: 1.2940 - val_accuracy: 0.4918
Epoch 5/10
15/15 [==============================] - 4s 262ms/step - loss: 1.1136 - accuracy: 0.4667 - val_loss: 1.0758 - val_accuracy: 0.4754
Epoch 6/10
15/15 [==============================] - 4s 259ms/step - loss: 0.9691 - accuracy: 0.5459 - val_loss: 1.0467 - val_accuracy: 0.5082
Epoch 7/10
15/15 [==============================] - 4s 232ms/step - loss: 0.9760 - accuracy: 0.5218 - val_loss: 0.9069 - val_accuracy: 0.5902
Epoch 

**vgg19**

In [ ]:
from tensorflow.keras.applications import VGG19

# Load pre-trained VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Add custom output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(unique_labels), activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model on training data
start_time = time.time()
history =model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=len(x_train) // batch_size, epochs=epochs, validation_data=(x_val, y_val))

end_time = time.time()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)

# Print metrics
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')

Epoch 1/10
15/15 [==============================] - 10s 338ms/step - loss: 1.3775 - accuracy: 0.3690 - val_loss: 0.9667 - val_accuracy: 0.4754
Epoch 2/10
15/15 [==============================] - 5s 288ms/step - loss: 1.0694 - accuracy: 0.5000 - val_loss: 0.9101 - val_accuracy: 0.5246
Epoch 3/10
15/15 [==============================] - 5s 301ms/step - loss: 0.9994 - accuracy: 0.5131 - val_loss: 0.8544 - val_accuracy: 0.5902
Epoch 4/10
15/15 [==============================] - 5s 297ms/step - loss: 0.9448 - accuracy: 0.5393 - val_loss: 0.9683 - val_accuracy: 0.5082
Epoch 5/10
15/15 [==============================] - 5s 297ms/step - loss: 0.9227 - accuracy: 0.5677 - val_loss: 0.8547 - val_accuracy: 0.5574
Epoch 6/10
15/15 [==============================] - 4s 265ms/step - loss: 0.8658 - accuracy: 0.6048 - val_loss: 0.8846 - val_accuracy: 0.5738
Epoch 7/10
15/15 [==============================] - 5s 311ms/step - loss: 0.8799 - accuracy: 0.5830 - val_loss: 0.8439 - val_accuracy: 0.5574
Epoch

**InceptionV3**

In [ ]:

import time
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))


# Add custom output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(unique_labels), activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model on training data
start_time = time.time()
history =model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=len(x_train) // batch_size, epochs=epochs, validation_data=(x_val, y_val))

end_time = time.time()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)

# Print metrics
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')

Epoch 1/10
15/15 [==============================] - 33s 278ms/step - loss: 1.5595 - accuracy: 0.4345 - val_loss: 4.8767 - val_accuracy: 0.3279
Epoch 2/10
15/15 [==============================] - 3s 219ms/step - loss: 1.0361 - accuracy: 0.5852 - val_loss: 3.9309 - val_accuracy: 0.3443
Epoch 3/10
15/15 [==============================] - 3s 193ms/step - loss: 0.7605 - accuracy: 0.6979 - val_loss: 3.5044 - val_accuracy: 0.3279
Epoch 4/10
15/15 [==============================] - 3s 189ms/step - loss: 0.6890 - accuracy: 0.6834 - val_loss: 2.7728 - val_accuracy: 0.4262
Epoch 5/10
15/15 [==============================] - 3s 194ms/step - loss: 0.5620 - accuracy: 0.7795 - val_loss: 2.4372 - val_accuracy: 0.4426
Epoch 6/10
15/15 [==============================] - 3s 188ms/step - loss: 0.5943 - accuracy: 0.7380 - val_loss: 1.8669 - val_accuracy: 0.5410
Epoch 7/10
15/15 [==============================] - 3s 189ms/step - loss: 0.5174 - accuracy: 0.7948 - val_loss: 1.2144 - val_accuracy: 0.6066
Epoch

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Define image size
image_size = 150

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, 0)
    image = cv2.bilateralFilter(image, 2, 50, 50)
    image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
    image = cv2.resize(image, (image_size, image_size))
    return image

# Define data directories
data_dirs = {
    'train': '/content/Data/train',
    'test': '/content/Data/test',
    'valid': '/content/Data/valid'
}

# Initialize lists to store data
data = {split: {'images': [], 'labels': []} for split in data_dirs}

# Iterate over data directories
for split, directory in data_dirs.items():
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for file in tqdm(os.listdir(label_dir), desc=f'Processing {split}/{label}'):
            image_path = os.path.join(label_dir, file)
            data[split]['images'].append(preprocess_image(image_path))
            data[split]['labels'].append(label)

# Convert lists to NumPy arrays and normalize images
for split in data:
    data[split]['images'] = np.array(data[split]['images']) / 255.0
    data[split]['labels'] = np.array(data[split]['labels'])

# Extract unique labels present in the data
unique_labels = np.unique(np.concatenate([data[split]['labels'] for split in data]))

# Convert labels to one-hot encoding using dynamically extracted labels
label_to_index = {label: i for i, label in enumerate(unique_labels)}
for split in data:
    data[split]['labels'] = np.array([label_to_index[label] for label in data[split]['labels']])

# Shuffle training data
data['train']['images'], data['train']['labels'] = shuffle(data['train']['images'], data['train']['labels'], random_state=42)

# Split dataset into training, validation, and test sets
train_val_split = 0.2
val_test_split = 0.5
x_train, x_val_test, y_train, y_val_test = train_test_split(data['train']['images'], data['train']['labels'], test_size=train_val_split, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=val_test_split, random_state=42)

# Apply data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
datagen.fit(x_train)

# Determine the number of output units based on the number of unique labels
num_classes = len(unique_labels)



Processing train/normal: 100%|██████████| 148/148 [00:02<00:00, 68.44it/s]
Processing train/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa: 100%|██████████| 155/155 [00:01<00:00, 139.82it/s]
Processing train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa: 100%|██████████| 115/115 [00:00<00:00, 141.19it/s]
Processing train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib: 100%|██████████| 195/195 [00:01<00:00, 141.85it/s]
Processing valid/normal: 100%|██████████| 13/13 [00:00<00:00, 52.47it/s]
Processing valid/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa: 100%|██████████| 15/15 [00:00<00:00, 155.04it/s]
Processing valid/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa: 100%|██████████| 21/21 [00:00<00:00, 155.77it/s]
Processing valid/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib: 100%|██████████| 23/23 [00:00<00:00, 155.58it/s]


In [10]:

print("Number of classes:", num_classes)


Number of classes: 7



,

 ,=1SDDDDDDDDDD

In [3]:
!pip install keras-tuner


In [21]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import kerastuner as kt
import numpy as np
from time import time

# Define Bayesian CNN model using Variational Inference
def build_model(hp):
    model = tf.keras.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten layer
    model.add(layers.Flatten())

    # Dense layers with variational dropout
    for i in range(hp.Int('num_layers', min_value=1, max_value=3, step=1)):
        model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                               activation='relu'))
        model.add(layers.Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)))

    # Output layer
    model.add(layers.Dense(num_classes))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4])),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

# Define a tuner
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='keras_tuner_dir',
                     project_name='bayesian_cnn')

# Search for the best hyperparameters
tuner.search(x_train, y_train,
             epochs=10,
             validation_data=(x_val, y_val))

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Fit the best model
start_time = time()
history = best_model.fit(x_train, y_train,
                         epochs=10,
                         validation_data=(x_val, y_val))
end_time = time()

# Evaluate the best model on test data
test_loss, test_accuracy = best_model.evaluate(x_test, y_test)

# Print metrics
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')


Trial 30 Complete [00h 00m 12s]
val_accuracy: 0.4754098355770111

Best val_accuracy So Far: 0.9180327653884888
Total elapsed time: 00h 08m 18s
Epoch 1/10
16/16 [==============================] - 4s 83ms/step - loss: 0.1676 - accuracy: 0.9510 - val_loss: 0.3365 - val_accuracy: 0.8525
Epoch 2/10
16/16 [==============================] - 1s 60ms/step - loss: 0.1276 - accuracy: 0.9551 - val_loss: 0.2463 - val_accuracy: 0.9180
Epoch 3/10
16/16 [==============================] - 1s 57ms/step - loss: 0.0754 - accuracy: 0.9837 - val_loss: 0.3465 - val_accuracy: 0.9016
Epoch 4/10
16/16 [==============================] - 1s 56ms/step - loss: 0.0415 - accuracy: 0.9857 - val_loss: 0.2521 - val_accuracy: 0.9180
Epoch 5/10
16/16 [==============================] - 1s 57ms/step - loss: 0.0730 - accuracy: 0.9980 - val_loss: 0.3100 - val_accuracy: 0.8689
Epoch 6/10
16/16 [==============================] - 1s 58ms/step - loss: 0.0325 - accuracy: 0.9939 - val_loss: 0.2229 - val_accuracy: 0.9344
Epoch 7/10
